<a href="https://colab.research.google.com/github/sahal-mulki/geo-llm/blob/main/geoclip_preds_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle

In [3]:
!kaggle datasets download -d sahalmulki/llms-for-image-geolocation-benchmarks

Dataset URL: https://www.kaggle.com/datasets/sahalmulki/llms-for-image-geolocation-benchmarks
License(s): MIT
100% 994M/996M [01:03<00:00, 18.0MB/s]
100% 996M/996M [01:04<00:00, 16.3MB/s]


In [4]:
!unzip llms-for-image-geolocation-benchmarks.zip -d /content/llms-for-image-geolocation-benchmarks

Streaming output truncated to the last 5000 lines.
  inflating: /content/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/508033358_5d1f3bc07a_221_8393530@N05.jpg  
  inflating: /content/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/508092310_ed7d7da28a_217_62126383@N00.jpg  
  inflating: /content/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/508344507_a1015d1cca_230_33463080@N00.jpg  
  inflating: /content/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/509419851_e5ce616c39_210_7412395@N07.jpg  
  inflating: /content/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/509733934_86d61a4c33_221_7885919@N07.jpg  
  inflating: /content/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/509757181_585351b261_192_7885919@N07.jpg  
  inflating: /content/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/509964802_63d13dac86_200_90168643@N00.jpg  
 

In [5]:
pip install geoclip

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
pip install -q geograpy3 gensim nltk ndg-httpsclient pyopenssl pyasn1

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
from tqdm import tqdm

In [ ]:
import os
import csv
import torch
from geoclip import GeoCLIP
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

model = GeoCLIP().to("cuda")
geolocator = Nominatim(user_agent="GEOCLIPOSINT")

folder_path = "/content/llms-for-image-geolocation-benchmarks/yfcc4k/yfcc4k/yfcc4k"
output_csv = "geoclip_predictions_yfcc.csv"

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Function to get address from coordinates
def get_address(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en', zoom=8)
        return location.address if location else "Address not found"
    except GeocoderTimedOut:
        location = geolocator.reverse((lat, lon), language='en', zoom=8)
        return location.address if location else "Address not found"

# Open a CSV file for writing
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    header = ["Image"]
    for i in range(1, 6):
        header.extend([f"Prediction {i} Latitude", f"Prediction {i} Longitude", f"Prediction {i} Probability", f"Prediction {i} Address"])
    writer.writerow(header)

    # Iterate through each image
    for image_file in tqdm(image_files):
        image_path = os.path.join(folder_path, image_file)

        top_pred_gps, top_pred_prob = model.predict(image_path, top_k=5)

        # Prepare a row for the current image
        row = [image_file]
        for i in range(5):
            lat, lon = top_pred_gps[i]
            lat = str(float(lat))
            lon = str(float(lon))
            prob = top_pred_prob[i]
            address = get_address(lat, lon)
            print(address)
            row.extend([lat, lon, prob, address])

        # Write the row to the CSV file
        writer.writerow(row)

print(f"Predictions saved to {output_csv}")


  0%|          | 0/4536 [00:00<?, ?it/s]

Midlothian, Scotland, United Kingdom
East Dunbartonshire, Scotland, United Kingdom
County Antrim, Northern Ireland, United Kingdom
East Dunbartonshire, Scotland, United Kingdom


  0%|          | 1/4536 [00:03<4:29:55,  3.57s/it]

East Ayrshire, Scotland, United Kingdom
Middlesex County, Massachusetts, United States
Middlesex County, Massachusetts, United States
Middlesex County, Massachusetts, United States
Middlesex County, Massachusetts, United States


  0%|          | 2/4536 [00:06<4:08:33,  3.29s/it]

...

Los Angeles County, California, United States
Lincolnshire, England, United Kingdom
Lincolnshire, England, United Kingdom
Lincolnshire, England, United Kingdom
Lincolnshire, England, United Kingdom


  6%|▌         | 270/4536 [13:16<3:28:05,  2.93s/it]

Forest County, Wisconsin, United States
Ito County, Wakayama Prefecture, Japan
Ito County, Wakayama Prefecture, Japan
Ito County, Wakayama Prefecture, Japan
Ito County, Wakayama Prefecture, Japan


  6%|▌         | 271/4536 [13:19<3:27:41,  2.92s/it]

Nara Prefecture, Japan
